In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/retail_features.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['sku_id','date']).copy()

In [2]:
demand_stats = df.groupby('sku_id')['demand'].agg(['mean','std']).reset_index()
demand_stats.columns = ['sku_id','avg_daily_demand','demand_std']

demand_stats.head()

,sku_id,avg_daily_demand,demand_std
0,SKU_1,73.778887,16.982458
1,SKU_10,41.502140,15.860969
2,SKU_11,96.757489,16.327399
3,SKU_12,63.248217,16.396812
4,SKU_13,85.242511,16.268672


In [3]:
np.random.seed(42)

demand_stats['lead_time_days'] = np.random.randint(3, 10, size=len(demand_stats))

In [4]:
Z = 1.65

demand_stats['safety_stock'] = (
    Z *
    demand_stats['demand_std'] *
    np.sqrt(demand_stats['lead_time_days'])
)

In [5]:
demand_stats['reorder_point'] = (
    demand_stats['avg_daily_demand'] *
    demand_stats['lead_time_days']
) + demand_stats['safety_stock']

In [6]:
demand_stats['current_stock'] = np.random.randint(50, 400, size=len(demand_stats))

In [7]:
demand_stats['reorder_flag'] = (
    demand_stats['current_stock'] <= demand_stats['reorder_point']
).astype(int)

In [8]:
S = 500
H = 2

demand_stats['annual_demand'] = demand_stats['avg_daily_demand'] * 365

demand_stats['EOQ'] = np.sqrt(
    (2 * demand_stats['annual_demand'] * S) / H
)

In [9]:
inventory_decisions = demand_stats[[
    'sku_id',
    'avg_daily_demand',
    'lead_time_days',
    'safety_stock',
    'reorder_point',
    'current_stock',
    'reorder_flag',
    'EOQ'
]]

inventory_decisions.head()

,sku_id,avg_daily_demand,lead_time_days,safety_stock,reorder_point,current_stock,reorder_flag,EOQ
0,SKU_1,73.778887,9,84.063170,748.073155,356,1,3669.420517
1,SKU_10,41.502140,6,64.104615,313.117454,184,1,2752.115643
2,SKU_11,96.757489,7,71.277092,748.579517,70,1,4202.171081
3,SKU_12,63.248217,9,81.164219,650.398171,378,1,3397.469584
4,SKU_13,85.242511,5,60.023462,486.236015,216,1,3944.205649


In [10]:
inventory_decisions.to_csv("../data/inventory_decisions.csv", index=False)

In [11]:
inventory_decisions['reorder_flag'].value_counts()

reorder_flag
1    41
0     9
Name: count, dtype: int64

In [12]:
future_forecast = pd.read_csv("../data/future_forecast.csv")
future_forecast['date'] = pd.to_datetime(future_forecast['date'])

In [13]:
forecast_stats = future_forecast.groupby('sku_id')['predicted_demand'] \
    .mean().reset_index()

forecast_stats.columns = ['sku_id', 'avg_forecast_demand']

In [14]:
inventory_decisions = inventory_decisions.merge(
    forecast_stats,
    on='sku_id',
    how='left'
)

In [15]:
inventory_decisions['reorder_point_forecast'] = (
    inventory_decisions['avg_forecast_demand'] *
    inventory_decisions['lead_time_days']
) + inventory_decisions['safety_stock']

In [16]:
inventory_decisions['reorder_flag_forecast'] = (
    inventory_decisions['current_stock'] <=
    inventory_decisions['reorder_point_forecast']
).astype(int)

In [17]:
# Assume 60% cost structure
inventory_decisions['unit_cost'] = inventory_decisions['avg_forecast_demand'] * 0  # placeholder

In [18]:
df['unit_cost'] = df['price'] * 0.6
df['profit_per_unit'] = df['price'] - df['unit_cost']
df['daily_profit'] = df['profit_per_unit'] * df['demand']

In [19]:
profit_stats = df.groupby('sku_id')['daily_profit'].mean().reset_index()
profit_stats.columns = ['sku_id', 'avg_daily_profit']

In [20]:
elasticity = -0.3

In [21]:
price_increase_pct = 0.05

inventory_decisions['new_demand_estimate'] = (
    inventory_decisions['avg_forecast_demand'] *
    (1 + elasticity * price_increase_pct)
)

In [22]:
promotion_uplift = 0.45

inventory_decisions['promo_demand_estimate'] = (
    inventory_decisions['avg_forecast_demand'] *
    (1 + promotion_uplift * 0.2)  # assume 20% promo increase
)

In [23]:
inventory_decisions['inventory_value'] = (
    inventory_decisions['current_stock'] *
    inventory_decisions['unit_cost']
)

In [24]:
inventory_decisions['capital_required'] = (
    inventory_decisions['EOQ'] *
    inventory_decisions['unit_cost']
)